## bash

In [5]:
!bash scripts/tune.sh

>>> LR=1e-3,DO=0.0,EL=2,DM=256,PL=16
True
Args in experiment:
Basic Config
  Task Name:          long_term_forecast  Is Training:        1                   
  Model ID:           EPF_168_24          Model:              TimeXer             

Data Loader
  Data:               custom              Root Path:          ./data              
  Data Path:          causal_data.csv     Features:           MS                  
  Target:             electricity_price   Freq:               h                   
  Checkpoints:        ./checkpoints/      
  Predictor(s):       solar_forecast,wind_forecast,total_load

Forecasting Task
  Seq Len:            168                 Label Len:          48                  
  Pred Len:           24                  Seasonal Patterns:  Monthly             
  Inverse:            0                   

Model Parameters
  Top k:              5                   Num Kernels:        6                   
  Enc In:             4                   Dec In:             4 

## df

In [23]:
import re, pandas as pd, pathlib, textwrap

LOG = pathlib.Path('tuning_saved.log')          # ← adjust if needed
ESC = re.compile(r'\x1b\[[0-9;]*m')             # strip ANSI colours

# patterns ------------------------------------------------------------
start_re   = re.compile(r'start training.*?_pl(\d+)_dm(\d+).*?_el(\d+)_')
lr_re      = re.compile(r'Learning Rate:\s*([0-9.eE+-]+)')
do_re      = re.compile(r'Dropout:\s*([0-9.eE+-]+)')
msemae_re  = re.compile(r'mse:\s*([0-9.eE+-]+).*?mae:\s*([0-9.eE+-]+)',
                        re.I)

rows, cur = [], {}

with LOG.open() as fh:
    for raw in fh:
        line = ESC.sub('', raw).strip()

        # -------- start-training line  (captures EL / DM / PL) ----------
        m = start_re.search(line)
        if m:
            cur.update({'PL': float(m.group(1)),
                        'DM': float(m.group(2)),
                        'EL': float(m.group(3))})
            continue

        # -------- Run-parameter lines  (captures LR / DO) ---------------
        m = lr_re.search(line)
        if m:
            cur['LR'] = float(m.group(1));  continue

        m = do_re.search(line)
        if m:
            cur['DO'] = float(m.group(1));  continue

        # -------- result line  (captures mse / mae → row) ---------------
        m = msemae_re.search(line)
        if m and {'LR','DO','EL','DM','PL'}.issubset(cur):
            rows.append({**cur,
                         'mse': float(m.group(1)),
                         'mae': float(m.group(2))})
            cur = {}                                    # reset for next run

# ------------------------------ dataframe -----------------------------
df = pd.DataFrame(rows)
if df.empty:
    raise RuntimeError(textwrap.dedent(f"""
       ⚠️  No rows parsed - check that {LOG} really contains the patterns
       “start training”, “Learning Rate:”, “Dropout:” and “mse:…”.
    """))

df = df[['LR','DO','EL','DM','PL','mse','mae']]

In [24]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

display(df.sort_values(['mse','mae']).reset_index(drop=True))

,LR,DO,EL,DM,PL,mse,mae
0,0.0005,0.2,2.0,256.0,24.0,0.104427,0.196775
1,0.0001,0.2,4.0,512.0,24.0,0.105916,0.195343
2,0.0001,0.2,4.0,256.0,24.0,0.106452,0.195598
3,0.0001,0.2,2.0,256.0,24.0,0.107361,0.196184
4,0.0001,0.2,3.0,256.0,24.0,0.107814,0.196356
5,0.0001,0.2,3.0,512.0,24.0,0.107995,0.197760
6,0.0001,0.1,2.0,512.0,24.0,0.108420,0.199717
7,0.0001,0.1,2.0,256.0,24.0,0.108478,0.199562
8,0.0001,0.2,2.0,512.0,24.0,0.108874,0.198089
9,0.0005,0.2,3.0,256.0,24.0,0.109037,0.199166


## winner

In [3]:
%%bash
bash scripts/TimeXer.sh \
     --features MS \
     --predictor solar_penetration,wind_penetration,total_load \
     --enc_in 4  --dec_in 4  --c_out 1 \
     --dropout 0.2  --learning_rate 5e-4

True
Args in experiment:
Basic Config
  Task Name:          long_term_forecast  Is Training:        1                   
  Model ID:           EPF_168_24          Model:              TimeXer             

Data Loader
  Data:               custom              Root Path:          ./data/             
  Data Path:          causal_data.csv     Features:           MS                  
  Target:             electricity_price   Freq:               h                   
  Checkpoints:        ./checkpoints/      
  Predictor(s):       solar_penetration,wind_penetration,total_load

Forecasting Task
  Seq Len:            168                 Label Len:          48                  
  Pred Len:           24                  Seasonal Patterns:  Monthly             
  Inverse:            0                   

Model Parameters
  Top k:              5                   Num Kernels:        6                   
  Enc In:             4                   Dec In:             4                   
  C Out:    

In [4]:
%%bash
bash scripts/TimeXer.sh \
     --features S \
     --enc_in 1  --dec_in 1  --c_out 1 \
     --dropout 0.2  --learning_rate 5e-4

True
Args in experiment:
Basic Config
  Task Name:          long_term_forecast  Is Training:        1                   
  Model ID:           EPF_168_24          Model:              TimeXer             

Data Loader
  Data:               custom              Root Path:          ./data/             
  Data Path:          causal_data.csv     Features:           S                   
  Target:             electricity_price   Freq:               h                   
  Checkpoints:        ./checkpoints/      
  Predictor(s):       total_load          

Forecasting Task
  Seq Len:            168                 Label Len:          48                  
  Pred Len:           24                  Seasonal Patterns:  Monthly             
  Inverse:            0                   

Model Parameters
  Top k:              5                   Num Kernels:        6                   
  Enc In:             1                   Dec In:             1                   
  C Out:              1              

In [8]:
%%bash
bash scripts/TimeXer.sh --features MS --predictor total_load --enc_in 2  --dec_in 2  --c_out 1 --dropout 0.2  --learning_rate 5e-4

True
Args in experiment:
Basic Config
  Task Name:          long_term_forecast  Is Training:        1                   
  Model ID:           EPF_168_24          Model:              TimeXer             

Data Loader
  Data:               custom              Root Path:          ./data/             
  Data Path:          causal_data.csv     Features:           MS                  
  Target:             electricity_price   Freq:               h                   
  Checkpoints:        ./checkpoints/      
  Predictor(s):       total_load          

Forecasting Task
  Seq Len:            168                 Label Len:          48                  
  Pred Len:           24                  Seasonal Patterns:  Monthly             
  Inverse:            0                   

Model Parameters
  Top k:              5                   Num Kernels:        6                   
  Enc In:             2                   Dec In:             2                   
  C Out:              1              